# Software para analise de treliça

In [1]:
from trelica import No, Barra
import numpy as np

In [2]:
# definindo elementos da barra

E = 210E9 # [Pa] módulo de elasticidade
A = 2E-4 # [m^2] área da seção transversal

In [3]:
# criando uma lista de 10 nós, sendo que todos estão no mesmo nivel em y, o primeiro travado em x e y
# e o ultimo com carga horizontal de kn

no_1 = No(x=0,y=0,x_lock=True,y_lock=False,x_carga=0,y_carga=0,x_numeracao_liberdade=0,y_numeracao_liberdade=1)
no_2 = No(x=0,y=0.4,x_lock=True,y_lock=True,x_carga=0,y_carga=0,x_numeracao_liberdade=2,y_numeracao_liberdade=3)
no_3 = No(x=0.3,y=0.4,x_lock=False,y_lock=False,x_carga=150,y_carga=-100,x_numeracao_liberdade=4,y_numeracao_liberdade=5)

lis_no = [no_1,no_2,no_3]

In [4]:
for no in lis_no:
    print(no.x, no.y, no.x_lock, no.y_lock, no.x_carga, no.y_carga, no.x_numeracao_liberdade, no.y_numeracao_liberdade)

0 0 True False 0 0 0 1
0 0.4 True True 0 0 2 3
0.3 0.4 False False 150 -100 4 5


In [5]:
# criando lista de elemento de barra

barra_1 = Barra(no_1,no_2,E,A)
barra_2 = Barra(no_2,no_3,E,A)
barra_3 = Barra(no_3,no_1,E,A)

lis_barra = [barra_1,barra_2,barra_3]

In [6]:
for barra in lis_barra:
    print(f"size = {barra.size}")

size = 0.4
size = 0.3
size = 0.5


## Definido a matrix de rigidez

In [7]:
import pandas as pd

In [8]:
matrix_rigidez = np.zeros(shape=(2*len(lis_no),2*len(lis_no)))
# matrix_rigidez_teste = np.zeros(shape=(2*len(lis_no),2*len(lis_no)))
matrix_rigidez

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [9]:
matrix_rigidez.shape

(6, 6)

In [10]:
for i in range(len(lis_barra)):
    barra = lis_barra[i]
    print(barra.k_local)
    # using np.ix_ to index the matrix
    matrix_rigidez[np.ix_(barra.dof,barra.dof)] += barra.k_local
    # matrix_rigidez_teste[barra.dof[0]:barra.dof[1]+1,barra.dof[0]:barra.dof[1]+1] += barra.k_local[:2,:2]
    # matrix_rigidez_teste[barra.dof[2]:barra.dof[3]+1,barra.dof[2]:barra.dof[3]+1] += barra.k_local[2:,2:]

[[ 0.00e+00  0.00e+00 -0.00e+00 -0.00e+00]
 [ 0.00e+00  1.05e+08 -0.00e+00 -1.05e+08]
 [-0.00e+00 -0.00e+00  0.00e+00  0.00e+00]
 [-0.00e+00 -1.05e+08  0.00e+00  1.05e+08]]
[[ 1.4e+08  0.0e+00 -1.4e+08 -0.0e+00]
 [ 0.0e+00  0.0e+00 -0.0e+00 -0.0e+00]
 [-1.4e+08 -0.0e+00  1.4e+08  0.0e+00]
 [-0.0e+00 -0.0e+00  0.0e+00  0.0e+00]]
[[ 30240000.          40320000.         -30240000.
  -40320000.        ]
 [ 40320000.          53760000.00000001 -40320000.
  -53760000.00000001]
 [-30240000.         -40320000.          30240000.
   40320000.        ]
 [-40320000.         -53760000.00000001  40320000.
   53760000.00000001]]


In [11]:
matrix_rigidez

array([[ 3.0240e+07,  4.0320e+07,  0.0000e+00,  0.0000e+00, -3.0240e+07,
        -4.0320e+07],
       [ 4.0320e+07,  1.5876e+08,  0.0000e+00, -1.0500e+08, -4.0320e+07,
        -5.3760e+07],
       [ 0.0000e+00,  0.0000e+00,  1.4000e+08,  0.0000e+00, -1.4000e+08,
         0.0000e+00],
       [ 0.0000e+00, -1.0500e+08,  0.0000e+00,  1.0500e+08,  0.0000e+00,
         0.0000e+00],
       [-3.0240e+07, -4.0320e+07, -1.4000e+08,  0.0000e+00,  1.7024e+08,
         4.0320e+07],
       [-4.0320e+07, -5.3760e+07,  0.0000e+00,  0.0000e+00,  4.0320e+07,
         5.3760e+07]])

## Criando vetor de forças

In [12]:
# para cada nó vamos verificar a carga aplicada e montar o vetor de carga

vector_carga = np.zeros(shape=(2*len(lis_no),1))

for no in lis_no:
    vector_carga[no.x_numeracao_liberdade] = no.x_carga
    vector_carga[no.y_numeracao_liberdade] = no.y_carga

In [13]:
vector_carga

array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [ 150.],
       [-100.]])

## Verificando os nós travados, e zerando os elementos da respectiva linha e coluna

In [14]:
# vamos agora aplicar as condições de contorno
# para cada nó verificar se ele está travado
# se sim, zerar todos os elementos com exeção do primeiro termo

for no in lis_no:
    if no.x_lock:
        # descobrindo o idx do nó
        idx = no.x_numeracao_liberdade
        # zerando todos os elementos 
        matrix_rigidez[idx,:] = 0
        matrix_rigidez[:,idx] = 0
        # colocando 1 em mat[idx,0]
        matrix_rigidez[idx,idx] = 1
        # zerando o valor do vetor de carga
        vector_carga[idx] = 0
    if no.y_lock:
        # descobrindo o idx do nó
        idx = no.y_numeracao_liberdade
        # zerando todos os elementos 
        matrix_rigidez[idx,:] = 0
        matrix_rigidez[:,idx] = 0
        # colocando o1 em mat[idx,0]
        matrix_rigidez[idx,idx] = 1
        # zerando o valor do vetor de carga
        vector_carga[idx] = 0

In [15]:
matrix_rigidez

array([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00],
       [ 0.0000e+00,  1.5876e+08,  0.0000e+00,  0.0000e+00, -4.0320e+07,
        -5.3760e+07],
       [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
         0.0000e+00],
       [ 0.0000e+00, -4.0320e+07,  0.0000e+00,  0.0000e+00,  1.7024e+08,
         4.0320e+07],
       [ 0.0000e+00, -5.3760e+07,  0.0000e+00,  0.0000e+00,  4.0320e+07,
         5.3760e+07]])

In [16]:
# calculando o determinante da matriz de rigidez
det = np.linalg.det(matrix_rigidez)
det

7.902720000000003e+23

In [17]:
vector_carga

array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [ 150.],
       [-100.]])

## Resolvendo o sistema linear

In [18]:
# o sistema matricial é dado por:
# K * u = F
# queremos saber u, logo:

u = np.linalg.solve(matrix_rigidez,vector_carga)
print(u)

[[ 0.00000000e+00]
 [-9.52380952e-07]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.60714286e-06]
 [-4.01785714e-06]]


Tensões em cada barra

In [36]:
for i in range(0,len(lis_barra)):
    barra = lis_barra[i]
    u_barra = u[barra.dof]
    e = (1/barra.size) * np.dot([-barra.cos, -barra.sin, barra.cos, barra.sin], u_barra)
    sigma = barra.modulo_elasticidade * e
    print(f"Barra {i+1}: tensão = {sigma} Pa")

Barra 1: tensão = [500000.] Pa
Barra 2: tensão = [1125000.] Pa
Barra 3: tensão = [-625000.] Pa
